# Module 08: Autostart Configuration

**Difficulty**: ⭐⭐  
**Estimated Time**: 45 minutes  
**Prerequisites**: 
- Module 02 (npm) OR Module 03 (Docker) completed
- n8n installed and working
- Administrator access to Windows

## Learning Objectives

By the end of this notebook, you will be able to:

1. Configure Windows Task Scheduler to auto-start n8n on system boot
2. Install and configure NSSM (Non-Sucking Service Manager) for service-based startup
3. Set up PM2 process manager for auto-restart and monitoring
4. Configure Docker Desktop auto-start with restart policies
5. Verify and troubleshoot auto-start configurations

## Why Auto-Start Matters

Manual startup is fine for development, but becomes problematic for production automation:

**Without auto-start**:
- After Windows updates: n8n doesn't restart
- After power outages: Workflows stop until manual restart
- After accidental reboots: Hours of downtime
- Vacation/travel: Can't restart remotely

**With auto-start**:
- System boots → n8n starts automatically
- Workflows resume without intervention
- Business continuity maintained
- Peace of mind

### Choosing the Right Method

| Method | Complexity | Reliability | Features | Best For |
|--------|------------|-------------|----------|----------|
| **Task Scheduler** | Low | High | Basic startup | npm users, simple needs |
| **NSSM** | Medium | Very High | Full Windows service | npm users, production |
| **PM2** | Medium | Medium | Monitoring, logs | Cross-platform needs |
| **Docker restart** | Low | High | Container-specific | Docker users |

**Recommendation**: 
- npm users: Start with Task Scheduler, upgrade to NSSM if needed
- Docker users: Use restart policies (simplest)

## Part 1: Windows Task Scheduler Setup

Task Scheduler is built into Windows and requires no additional software.

### Advantages

✅ No additional software needed  
✅ Native Windows integration  
✅ Easy to configure  
✅ Visual interface available  
✅ Detailed logging  

### Disadvantages

❌ Not a true Windows service  
❌ Visible console window (can minimize)  
❌ Requires login on some configurations  

### Step-by-Step: GUI Method

#### 1. Create Startup Batch Script

First, create `C:\n8n\start-n8n.bat`:

```batch
@echo off
REM n8n Auto-Start Script
REM This script runs n8n with configured environment variables

REM Change to n8n directory
cd /d C:\n8n

REM Set environment variables
SET N8N_PORT=5678
SET N8N_BASIC_AUTH_ACTIVE=true
SET N8N_BASIC_AUTH_USER=admin
SET N8N_BASIC_AUTH_PASSWORD=your_secure_password
SET GENERIC_TIMEZONE=America/New_York
SET EXECUTIONS_DATA_PRUNE=true
SET EXECUTIONS_DATA_MAX_AGE=168
SET N8N_DEFAULT_BINARY_DATA_MODE=filesystem

REM Optional: Memory limit
SET NODE_OPTIONS=--max-old-space-size=2048

REM Log startup
echo [%DATE% %TIME%] Starting n8n... >> C:\n8n\logs\startup.log

REM Start n8n
n8n
```

#### 2. Test Batch Script

Double-click `start-n8n.bat` to verify it works before scheduling.

#### 3. Open Task Scheduler

- Press `Win + R`
- Type `taskschd.msc`
- Press Enter

#### 4. Create New Task

- Click "Create Task" (not "Create Basic Task")
- Name: `n8n Automation Server`
- Description: `Auto-start n8n workflow automation on system boot`

#### 5. General Tab Settings

- ✅ Check "Run whether user is logged on or not"
- ✅ Check "Run with highest privileges"
- ✅ Check "Hidden" (optional - hides console window)
- Configure for: "Windows 10"

#### 6. Triggers Tab

- Click "New"
- Begin the task: "At startup"
- Delay task for: `1 minute` (gives Windows time to initialize)
- ✅ Check "Enabled"
- Click OK

#### 7. Actions Tab

- Click "New"
- Action: "Start a program"
- Program/script: `C:\n8n\start-n8n.bat`
- Start in: `C:\n8n`
- Click OK

#### 8. Conditions Tab

- ❌ UNCHECK "Start the task only if the computer is on AC power"
- ❌ UNCHECK "Stop if the computer switches to battery power"
- (Critical for laptops!)

#### 9. Settings Tab

- ✅ Check "Allow task to be run on demand"
- ✅ Check "Run task as soon as possible after a scheduled start is missed"
- ❌ UNCHECK "Stop the task if it runs longer than"
- If the running task does not end when requested: "Do not start a new instance"

#### 10. Save and Test

- Click OK
- Enter your Windows password when prompted
- Right-click the task → Run
- Verify n8n starts
- Restart computer to test auto-start

### PowerShell Method (Faster)

```powershell
# Run as Administrator

$action = New-ScheduledTaskAction -Execute "C:\n8n\start-n8n.bat" -WorkingDirectory "C:\n8n"
$trigger = New-ScheduledTaskTrigger -AtStartup -RandomDelay (New-TimeSpan -Seconds 60)
$principal = New-ScheduledTaskPrincipal -UserId "SYSTEM" -RunLevel Highest
$settings = New-ScheduledTaskSettingsSet -AllowStartIfOnBatteries -DontStopIfGoingOnBatteries -ExecutionTimeLimit 0

Register-ScheduledTask -TaskName "n8n-autostart" `
  -Action $action `
  -Trigger $trigger `
  -Principal $principal `
  -Settings $settings `
  -Description "Auto-start n8n workflow automation"

# Test the task
Start-ScheduledTask -TaskName "n8n-autostart"
```

### Viewing Logs

```powershell
# View task history
Get-ScheduledTaskInfo -TaskName "n8n-autostart"

# View last run result
(Get-ScheduledTask -TaskName "n8n-autostart").State
```

In [ ]:
# Task Scheduler configuration generator

import os
from pathlib import Path

def generate_task_scheduler_config(n8n_path="C:\\n8n", delay_seconds=60):
    """
    Generate PowerShell script to create Task Scheduler task.
    
    Args:
        n8n_path: Path to n8n installation directory
        delay_seconds: Delay after startup before starting n8n
    """
    print("=" * 70)
    print("TASK SCHEDULER CONFIGURATION GENERATOR")
    print("=" * 70)
    
    batch_script = f"""@echo off
REM n8n Auto-Start Script
REM Generated by n8n Windows Automation Course

REM Change to n8n directory
cd /d {n8n_path}

REM Create logs directory if it doesn't exist
if not exist "{n8n_path}\\logs" mkdir "{n8n_path}\\logs"

REM Set environment variables
SET N8N_PORT=5678
SET N8N_BASIC_AUTH_ACTIVE=true
SET N8N_BASIC_AUTH_USER=admin
SET N8N_BASIC_AUTH_PASSWORD=CHANGE_THIS_PASSWORD
SET GENERIC_TIMEZONE=America/New_York
SET EXECUTIONS_DATA_PRUNE=true
SET EXECUTIONS_DATA_MAX_AGE=168
SET N8N_DEFAULT_BINARY_DATA_MODE=filesystem
SET NODE_OPTIONS=--max-old-space-size=2048

REM Log startup
echo [%DATE% %TIME%] Starting n8n auto-start... >> "{n8n_path}\\logs\\startup.log"

REM Start n8n
echo Starting n8n...
n8n
"""
    
    powershell_script = f"""# PowerShell script to create Task Scheduler task for n8n
# Run this script as Administrator

Write-Host "Creating Task Scheduler task for n8n..." -ForegroundColor Green

# Define task components
$action = New-ScheduledTaskAction `
  -Execute "{n8n_path}\\start-n8n.bat" `
  -WorkingDirectory "{n8n_path}"

$trigger = New-ScheduledTaskTrigger `
  -AtStartup `
  -RandomDelay (New-TimeSpan -Seconds {delay_seconds})

$principal = New-ScheduledTaskPrincipal `
  -UserId "SYSTEM" `
  -RunLevel Highest

$settings = New-ScheduledTaskSettingsSet `
  -AllowStartIfOnBatteries `
  -DontStopIfGoingOnBatteries `
  -DontStopOnIdleEnd `
  -ExecutionTimeLimit (New-TimeSpan -Hours 0)

# Register the task
try {{
    # Remove existing task if it exists
    Unregister-ScheduledTask -TaskName "n8n-autostart" -Confirm:$false -ErrorAction SilentlyContinue
    
    # Register new task
    Register-ScheduledTask `
      -TaskName "n8n-autostart" `
      -Action $action `
      -Trigger $trigger `
      -Principal $principal `
      -Settings $settings `
      -Description "Auto-start n8n workflow automation server"
    
    Write-Host "✅ Task created successfully!" -ForegroundColor Green
    Write-Host ""
    Write-Host "To test: Start-ScheduledTask -TaskName 'n8n-autostart'" -ForegroundColor Yellow
    Write-Host "To remove: Unregister-ScheduledTask -TaskName 'n8n-autostart'" -ForegroundColor Yellow
    
}} catch {{
    Write-Host "❌ Error creating task: $_" -ForegroundColor Red
}}
"""
    
    print(f"\n📂 Installation Path: {n8n_path}")
    print(f"⏱️  Startup Delay: {delay_seconds} seconds")
    
    print("\n" + "─" * 70)
    print("STEP 1: CREATE BATCH SCRIPT")
    print("─" * 70)
    print(f"\nSave this as: {n8n_path}\\start-n8n.bat")
    print("\n" + batch_script)
    
    print("\n" + "─" * 70)
    print("STEP 2: CREATE TASK SCHEDULER TASK")
    print("─" * 70)
    print(f"\nSave this as: {n8n_path}\\create-task.ps1")
    print("\n" + powershell_script)
    
    print("\n" + "─" * 70)
    print("STEP 3: RUN THE POWERSHELL SCRIPT")
    print("─" * 70)
    print("\n1. Open PowerShell as Administrator")
    print("2. Navigate to n8n directory:")
    print(f"   cd {n8n_path}")
    print("3. Run the script:")
    print("   .\\create-task.ps1")
    print("\n   If you get execution policy error:")
    print("   Set-ExecutionPolicy -ExecutionPolicy RemoteSigned -Scope CurrentUser")
    
    print("\n" + "─" * 70)
    print("STEP 4: TEST THE TASK")
    print("─" * 70)
    print("\n1. Manually trigger:")
    print("   Start-ScheduledTask -TaskName 'n8n-autostart'")
    print("\n2. Check if n8n is running:")
    print("   Open browser → http://localhost:5678")
    print("\n3. Restart computer to test auto-start")
    
    print("\n" + "─" * 70)
    print("TROUBLESHOOTING")
    print("─" * 70)
    print("\nView task status:")
    print("  Get-ScheduledTask -TaskName 'n8n-autostart'")
    print("\nView last run result:")
    print("  Get-ScheduledTaskInfo -TaskName 'n8n-autostart'")
    print("\nRemove task:")
    print("  Unregister-ScheduledTask -TaskName 'n8n-autostart' -Confirm:$false")
    
    print("\n" + "=" * 70)
    
    return batch_script, powershell_script

# Generate configuration
batch, ps1 = generate_task_scheduler_config("C:\\n8n", 60)

## Part 2: NSSM (Non-Sucking Service Manager)

NSSM converts any application into a proper Windows service.

### Advantages

✅ True Windows service  
✅ No console window  
✅ Automatic restart on failure  
✅ Runs before user login  
✅ Service management integration  
✅ Better than Task Scheduler for production  

### Disadvantages

❌ Requires downloading third-party tool  
❌ Slightly more complex setup  
❌ Command-line configuration  

### Installation Steps

#### 1. Download NSSM

- Visit: https://nssm.cc/download
- Download latest version (e.g., `nssm-2.24.zip`)
- Extract to `C:\tools\nssm\`

#### 2. Add to PATH

```powershell
# Add NSSM to system PATH (run as Administrator)
$path = [Environment]::GetEnvironmentVariable('Path', 'Machine')
$path += ';C:\tools\nssm\win64'
[Environment]::SetEnvironmentVariable('Path', $path, 'Machine')

# Restart terminal to apply
```

Or manually:
1. Win + R → `sysdm.cpl` → Enter
2. Advanced → Environment Variables
3. System variables → Path → Edit
4. New → `C:\tools\nssm\win64`
5. OK → OK → OK
6. Restart terminal

#### 3. Install n8n as Service (GUI Method)

```powershell
# Run as Administrator
nssm install n8n
```

This opens a GUI:
- **Application tab**:
  - Path: `C:\Program Files\nodejs\node.exe`
  - Startup directory: `C:\n8n`
  - Arguments: `C:\Users\YourUsername\AppData\Roaming\npm\node_modules\n8n\bin\n8n`

- **Details tab**:
  - Display name: `n8n Workflow Automation`
  - Description: `n8n workflow automation server`
  - Startup type: `Automatic`

- **Environment tab**:
  ```
  N8N_PORT=5678
  N8N_BASIC_AUTH_ACTIVE=true
  N8N_BASIC_AUTH_USER=admin
  N8N_BASIC_AUTH_PASSWORD=your_password
  GENERIC_TIMEZONE=America/New_York
  EXECUTIONS_DATA_PRUNE=true
  EXECUTIONS_DATA_MAX_AGE=168
  ```

- **I/O tab**:
  - Output (stdout): `C:\n8n\logs\n8n-output.log`
  - Error (stderr): `C:\n8n\logs\n8n-error.log`

- **Rotation tab**:
  - ✅ Check "Rotate files"
  - Restrict rotation: `10485760` (10MB)

Click "Install service"

#### 4. Install n8n as Service (Command-Line Method)

```powershell
# Run as Administrator

# Install service
nssm install n8n "C:\Program Files\nodejs\node.exe"

# Set n8n path as argument
$n8nPath = "$env:APPDATA\npm\node_modules\n8n\bin\n8n"
nssm set n8n AppParameters $n8nPath

# Set working directory
nssm set n8n AppDirectory "C:\n8n"

# Set display name
nssm set n8n DisplayName "n8n Workflow Automation"

# Set description
nssm set n8n Description "n8n workflow automation server"

# Set startup type
nssm set n8n Start SERVICE_AUTO_START

# Set environment variables
nssm set n8n AppEnvironmentExtra N8N_PORT=5678 N8N_BASIC_AUTH_ACTIVE=true N8N_BASIC_AUTH_USER=admin N8N_BASIC_AUTH_PASSWORD=your_password GENERIC_TIMEZONE=America/New_York EXECUTIONS_DATA_PRUNE=true EXECUTIONS_DATA_MAX_AGE=168

# Set log files
nssm set n8n AppStdout "C:\n8n\logs\n8n-output.log"
nssm set n8n AppStderr "C:\n8n\logs\n8n-error.log"

# Enable log rotation (10MB max)
nssm set n8n AppStdoutCreationDisposition 4
nssm set n8n AppStderrCreationDisposition 4
nssm set n8n AppRotateFiles 1
nssm set n8n AppRotateBytes 10485760

# Start the service
nssm start n8n
```

#### 5. Manage Service

```powershell
# Start service
nssm start n8n
# OR: Start-Service n8n

# Stop service
nssm stop n8n
# OR: Stop-Service n8n

# Restart service
nssm restart n8n
# OR: Restart-Service n8n

# Check status
nssm status n8n
# OR: Get-Service n8n

# View configuration
nssm dump n8n

# Edit service
nssm edit n8n

# Remove service
nssm stop n8n
nssm remove n8n confirm
```

#### 6. View in Services

- Press `Win + R`
- Type `services.msc`
- Find "n8n Workflow Automation"
- Right-click → Properties → Startup type: Automatic

### Auto-Restart on Failure

NSSM automatically handles restarts. Configure recovery:

```powershell
# Set restart behavior
nssm set n8n AppExit Default Restart

# Restart after 5 seconds on any exit
nssm set n8n AppThrottle 5000
```

Or in Services.msc:
1. Right-click service → Properties
2. Recovery tab
3. First failure: Restart the Service
4. Second failure: Restart the Service
5. Subsequent failures: Restart the Service
6. Restart service after: 1 minutes

In [ ]:
# NSSM configuration generator

import os

def generate_nssm_config():
    """
    Generate PowerShell script to install n8n as Windows service using NSSM.
    """
    print("=" * 70)
    print("NSSM SERVICE CONFIGURATION GENERATOR")
    print("=" * 70)
    
    # Get common paths
    node_path = "C:\\Program Files\\nodejs\\node.exe"
    appdata = os.environ.get('APPDATA', 'C:\\Users\\YourUsername\\AppData\\Roaming')
    n8n_bin = f"{appdata}\\npm\\node_modules\\n8n\\bin\\n8n"
    
    install_script = f"""# NSSM Service Installation Script for n8n
# Run this script as Administrator in PowerShell

Write-Host "Installing n8n as Windows Service using NSSM..." -ForegroundColor Green
Write-Host ""

# Check if NSSM is installed
if (-not (Get-Command nssm -ErrorAction SilentlyContinue)) {{
    Write-Host "❌ NSSM not found!" -ForegroundColor Red
    Write-Host ""
    Write-Host "Please install NSSM first:" -ForegroundColor Yellow
    Write-Host "1. Download from: https://nssm.cc/download"
    Write-Host "2. Extract to C:\\tools\\nssm"
    Write-Host "3. Add to PATH: C:\\tools\\nssm\\win64"
    exit 1
}}

Write-Host "✅ NSSM found" -ForegroundColor Green

# Check if node.exe exists
if (-not (Test-Path "{node_path}")) {{
    Write-Host "❌ Node.js not found at {node_path}" -ForegroundColor Red
    Write-Host "Please install Node.js first" -ForegroundColor Yellow
    exit 1
}}

Write-Host "✅ Node.js found" -ForegroundColor Green

# Check if n8n is installed
if (-not (Test-Path "{n8n_bin}")) {{
    Write-Host "❌ n8n not found at {n8n_bin}" -ForegroundColor Red
    Write-Host "Please install n8n first: npm install -g n8n" -ForegroundColor Yellow
    exit 1
}}

Write-Host "✅ n8n found" -ForegroundColor Green
Write-Host ""

# Create logs directory
if (-not (Test-Path "C:\\n8n\\logs")) {{
    New-Item -Path "C:\\n8n\\logs" -ItemType Directory -Force | Out-Null
    Write-Host "✅ Created logs directory" -ForegroundColor Green
}}

# Remove existing service if it exists
try {{
    $serviceExists = Get-Service -Name n8n -ErrorAction SilentlyContinue
    if ($serviceExists) {{
        Write-Host "Removing existing n8n service..." -ForegroundColor Yellow
        nssm stop n8n
        nssm remove n8n confirm
    }}
}} catch {{}}

Write-Host ""
Write-Host "Installing n8n service..." -ForegroundColor Cyan

# Install service
nssm install n8n "{node_path}"

# Set n8n path
nssm set n8n AppParameters "{n8n_bin}"

# Set working directory
nssm set n8n AppDirectory "C:\\n8n"

# Set display name and description
nssm set n8n DisplayName "n8n Workflow Automation"
nssm set n8n Description "n8n workflow automation server"

# Set startup type to automatic
nssm set n8n Start SERVICE_AUTO_START

# Set environment variables
nssm set n8n AppEnvironmentExtra `
  "N8N_PORT=5678" `
  "N8N_BASIC_AUTH_ACTIVE=true" `
  "N8N_BASIC_AUTH_USER=admin" `
  "N8N_BASIC_AUTH_PASSWORD=CHANGE_THIS_PASSWORD" `
  "GENERIC_TIMEZONE=America/New_York" `
  "EXECUTIONS_DATA_PRUNE=true" `
  "EXECUTIONS_DATA_MAX_AGE=168" `
  "N8N_DEFAULT_BINARY_DATA_MODE=filesystem" `
  "NODE_OPTIONS=--max-old-space-size=2048"

# Set log files
nssm set n8n AppStdout "C:\\n8n\\logs\\n8n-output.log"
nssm set n8n AppStderr "C:\\n8n\\logs\\n8n-error.log"

# Enable log rotation (10MB max per file)
nssm set n8n AppStdoutCreationDisposition 4
nssm set n8n AppStderrCreationDisposition 4
nssm set n8n AppRotateFiles 1
nssm set n8n AppRotateBytes 10485760

# Set restart behavior
nssm set n8n AppExit Default Restart
nssm set n8n AppThrottle 5000

Write-Host ""
Write-Host "✅ Service installed successfully!" -ForegroundColor Green
Write-Host ""

# Start the service
Write-Host "Starting n8n service..." -ForegroundColor Cyan
nssm start n8n

Start-Sleep -Seconds 3

# Check status
$status = nssm status n8n
if ($status -eq "SERVICE_RUNNING") {{
    Write-Host "✅ n8n service is RUNNING" -ForegroundColor Green
    Write-Host ""
    Write-Host "Access n8n at: http://localhost:5678" -ForegroundColor Yellow
    Write-Host ""
    Write-Host "Useful commands:" -ForegroundColor Cyan
    Write-Host "  Start:   nssm start n8n"
    Write-Host "  Stop:    nssm stop n8n"
    Write-Host "  Restart: nssm restart n8n"
    Write-Host "  Status:  nssm status n8n"
    Write-Host "  Edit:    nssm edit n8n"
    Write-Host "  Logs:    Get-Content C:\\n8n\\logs\\n8n-output.log -Tail 50"
}} else {{
    Write-Host "⚠️  Service installed but not running" -ForegroundColor Yellow
    Write-Host "Check logs: C:\\n8n\\logs\\n8n-error.log" -ForegroundColor Yellow
}}
"""
    
    print("\n📝 INSTALLATION INSTRUCTIONS:")
    print("\n1. Ensure NSSM is installed:")
    print("   • Download from: https://nssm.cc/download")
    print("   • Extract to: C:\\tools\\nssm")
    print("   • Add to PATH: C:\\tools\\nssm\\win64")
    
    print("\n2. Save the script below as: C:\\n8n\\install-service.ps1")
    print("\n3. Run PowerShell as Administrator")
    print("\n4. Execute: C:\\n8n\\install-service.ps1")
    
    print("\n" + "─" * 70)
    print("POWERSHELL SCRIPT:")
    print("─" * 70)
    print(install_script)
    
    print("\n" + "─" * 70)
    print("POST-INSTALLATION:")
    print("─" * 70)
    print("\n1. Verify service is running:")
    print("   Get-Service n8n")
    print("\n2. View in Services GUI:")
    print("   services.msc")
    print("\n3. Test n8n access:")
    print("   http://localhost:5678")
    print("\n4. View logs:")
    print("   Get-Content C:\\n8n\\logs\\n8n-output.log -Tail 50 -Wait")
    
    print("\n" + "=" * 70)
    
    return install_script

# Generate NSSM configuration
nssm_script = generate_nssm_config()

## Part 3: PM2 Process Manager (Cross-Platform Option)

PM2 is a Node.js process manager with built-in monitoring.

### Advantages

✅ Built-in monitoring dashboard  
✅ Automatic restart on crash  
✅ Log management  
✅ Cross-platform (works on Linux too)  
✅ Memory/CPU monitoring  

### Disadvantages

❌ Windows auto-start requires Task Scheduler anyway  
❌ More complex than needed for Windows-only use  
❌ Extra memory overhead  

### Installation

```powershell
# Install PM2 globally
npm install -g pm2

# Verify installation
pm2 --version
```

### Create Ecosystem File

Create `C:\n8n\ecosystem.config.js`:

```javascript
module.exports = {
  apps: [{
    name: 'n8n',
    script: 'n8n',
    args: 'start',
    cwd: 'C:\\n8n',
    env: {
      N8N_PORT: 5678,
      N8N_BASIC_AUTH_ACTIVE: 'true',
      N8N_BASIC_AUTH_USER: 'admin',
      N8N_BASIC_AUTH_PASSWORD: 'your_password',
      GENERIC_TIMEZONE: 'America/New_York',
      EXECUTIONS_DATA_PRUNE: 'true',
      EXECUTIONS_DATA_MAX_AGE: '168',
      N8N_DEFAULT_BINARY_DATA_MODE: 'filesystem',
      NODE_OPTIONS: '--max-old-space-size=2048'
    },
    autorestart: true,
    watch: false,
    max_memory_restart: '1G',
    error_file: 'C:\\n8n\\logs\\pm2-error.log',
    out_file: 'C:\\n8n\\logs\\pm2-output.log',
    log_date_format: 'YYYY-MM-DD HH:mm:ss Z',
    merge_logs: true,
    max_restarts: 10,
    min_uptime: '10s'
  }]
};
```

### Start with PM2

```powershell
# Start n8n
pm2 start C:\n8n\ecosystem.config.js

# Save PM2 process list
pm2 save

# View status
pm2 status

# View logs
pm2 logs n8n

# View monitoring dashboard
pm2 monit
```

### Auto-Start with Task Scheduler

PM2's `pm2-startup` doesn't work well on Windows. Use Task Scheduler instead:

Create `C:\n8n\start-pm2.bat`:

```batch
@echo off
REM Resurrect PM2 processes
pm2 resurrect
```

Then create Task Scheduler task (as shown in Part 1) pointing to this batch file.

### PM2 Management Commands

```powershell
# List processes
pm2 list

# Stop n8n
pm2 stop n8n

# Restart n8n
pm2 restart n8n

# Delete from PM2
pm2 delete n8n

# View real-time logs
pm2 logs n8n --lines 100

# Clear logs
pm2 flush

# Monitor CPU/Memory
pm2 monit
```

## Part 4: Docker Desktop Auto-Start

For Docker installations, auto-start involves two components:

1. Docker Desktop itself
2. n8n container restart policy

### Configure Docker Desktop Auto-Start

#### Method 1: Docker Desktop Settings

1. Open Docker Desktop
2. Click Settings (gear icon)
3. General tab
4. ✅ Check "Start Docker Desktop when you log in"
5. Click "Apply & Restart"

#### Method 2: Task Scheduler (More Reliable)

```powershell
# Create Task Scheduler task for Docker Desktop
$action = New-ScheduledTaskAction `
  -Execute "C:\Program Files\Docker\Docker\Docker Desktop.exe"

$trigger = New-ScheduledTaskTrigger `
  -AtStartup `
  -RandomDelay (New-TimeSpan -Minutes 2)

$principal = New-ScheduledTaskPrincipal `
  -UserId $env:USERNAME `
  -RunLevel Highest

$settings = New-ScheduledTaskSettingsSet `
  -AllowStartIfOnBatteries `
  -DontStopIfGoingOnBatteries

Register-ScheduledTask `
  -TaskName "Docker-Desktop-Startup" `
  -Action $action `
  -Trigger $trigger `
  -Principal $principal `
  -Settings $settings `
  -Description "Start Docker Desktop on system boot"
```

**Note**: 2-minute delay allows Windows to complete startup before Docker initializes.

### Configure Container Restart Policy

#### docker run Method

```powershell
docker run -d `
  --name n8n `
  -p 5678:5678 `
  -v n8n_data:/home/node/.n8n `
  --restart unless-stopped `
  docker.n8n.io/n8nio/n8n
```

#### docker-compose.yml Method

```yaml
services:
  n8n:
    image: docker.n8n.io/n8nio/n8n
    container_name: n8n
    ports:
      - "5678:5678"
    environment:
      - GENERIC_TIMEZONE=America/New_York
      - N8N_BASIC_AUTH_ACTIVE=true
      - N8N_BASIC_AUTH_USER=admin
      - N8N_BASIC_AUTH_PASSWORD=secure_password
    volumes:
      - n8n_data:/home/node/.n8n
    restart: unless-stopped  # ← THIS LINE

volumes:
  n8n_data:
```

### Restart Policy Options

| Policy | Behavior | Use Case |
|--------|----------|----------|
| `no` | Never restart | Development/testing |
| `on-failure` | Restart only on error | May miss some failures |
| `always` | Always restart | Even after manual stop |
| `unless-stopped` | Restart unless manually stopped | **Recommended** |

### Testing Auto-Start

1. Ensure Docker Desktop starts on boot (check Task Scheduler or Docker settings)
2. Ensure container has `restart: unless-stopped`
3. Restart computer
4. Wait 3-5 minutes for Docker to fully initialize
5. Check if n8n is accessible: http://localhost:5678

### Troubleshooting Docker Auto-Start

```powershell
# Check if Docker Desktop is running
Get-Process "Docker Desktop" -ErrorAction SilentlyContinue

# Check container status
docker ps -a

# View container logs
docker logs n8n

# Manually start container
docker start n8n

# Check restart policy
docker inspect n8n --format '{{.HostConfig.RestartPolicy.Name}}'

# Update restart policy on existing container
docker update --restart unless-stopped n8n
```

In [ ]:
# Auto-start verification tool

import subprocess
import socket
import time

def verify_autostart_config(installation_type='npm'):
    """
    Verify auto-start configuration for n8n.
    
    Args:
        installation_type: 'npm', 'docker', or 'both'
    """
    print("=" * 70)
    print("AUTO-START CONFIGURATION VERIFICATION")
    print("=" * 70)
    
    results = {
        'checks_passed': 0,
        'checks_failed': 0,
        'checks_warning': 0
    }
    
    # Check 1: n8n is currently running
    print("\n1️⃣  Checking if n8n is currently accessible...")
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(2)
    n8n_running = sock.connect_ex(('localhost', 5678)) == 0
    sock.close()
    
    if n8n_running:
        print("   ✅ n8n is accessible on port 5678")
        results['checks_passed'] += 1
    else:
        print("   ❌ n8n is NOT accessible")
        print("   → Start n8n before testing auto-start configuration")
        results['checks_failed'] += 1
    
    # Check 2: Task Scheduler (for npm)
    if installation_type in ['npm', 'both']:
        print("\n2️⃣  Checking Task Scheduler configuration...")
        try:
            result = subprocess.run(
                ['powershell', '-Command', 
                 'Get-ScheduledTask -TaskName "n8n*" -ErrorAction SilentlyContinue'],
                capture_output=True,
                text=True,
                timeout=5
            )
            
            if 'n8n' in result.stdout.lower():
                print("   ✅ Task Scheduler task found")
                results['checks_passed'] += 1
                
                # Check if enabled
                if 'Ready' in result.stdout or 'Running' in result.stdout:
                    print("   ✅ Task is enabled")
                    results['checks_passed'] += 1
                else:
                    print("   ⚠️  Task may be disabled")
                    results['checks_warning'] += 1
            else:
                print("   ❌ No Task Scheduler task found")
                print("   → Create task using Part 1 instructions")
                results['checks_failed'] += 1
        except Exception as e:
            print(f"   ⚠️  Could not check Task Scheduler: {e}")
            results['checks_warning'] += 1
    
    # Check 3: Windows Service (NSSM)
    if installation_type in ['npm', 'both']:
        print("\n3️⃣  Checking for Windows Service (NSSM)...")
        try:
            result = subprocess.run(
                ['powershell', '-Command',
                 'Get-Service -Name "n8n" -ErrorAction SilentlyContinue'],
                capture_output=True,
                text=True,
                timeout=5
            )
            
            if 'n8n' in result.stdout.lower():
                print("   ✅ n8n Windows Service found")
                results['checks_passed'] += 1
                
                if 'Running' in result.stdout:
                    print("   ✅ Service is running")
                    results['checks_passed'] += 1
                else:
                    print("   ⚠️  Service exists but not running")
                    results['checks_warning'] += 1
            else:
                print("   ℹ️  No Windows Service found (optional)")
        except Exception as e:
            print(f"   ℹ️  Could not check services: {e}")
    
    # Check 4: Docker Desktop (for Docker)
    if installation_type in ['docker', 'both']:
        print("\n4️⃣  Checking Docker Desktop...")
        try:
            result = subprocess.run(
                ['docker', '--version'],
                capture_output=True,
                text=True,
                timeout=5
            )
            
            if result.returncode == 0:
                print("   ✅ Docker is installed and accessible")
                results['checks_passed'] += 1
                
                # Check container restart policy
                result = subprocess.run(
                    ['docker', 'inspect', 'n8n', '--format', 
                     '{{.HostConfig.RestartPolicy.Name}}'],
                    capture_output=True,
                    text=True,
                    timeout=5
                )
                
                restart_policy = result.stdout.strip()
                if 'unless-stopped' in restart_policy or 'always' in restart_policy:
                    print(f"   ✅ Container restart policy: {restart_policy}")
                    results['checks_passed'] += 1
                else:
                    print(f"   ⚠️  Restart policy: {restart_policy}")
                    print("   → Recommended: unless-stopped")
                    results['checks_warning'] += 1
            else:
                print("   ❌ Docker not accessible")
                results['checks_failed'] += 1
        except Exception as e:
            print(f"   ❌ Docker error: {e}")
            results['checks_failed'] += 1
    
    # Summary
    print("\n" + "=" * 70)
    print("VERIFICATION SUMMARY")
    print("=" * 70)
    
    total_checks = results['checks_passed'] + results['checks_failed'] + results['checks_warning']
    
    print(f"\n✅ Passed:  {results['checks_passed']}")
    print(f"⚠️  Warnings: {results['checks_warning']}")
    print(f"❌ Failed:  {results['checks_failed']}")
    
    if results['checks_failed'] == 0 and results['checks_warning'] == 0:
        print("\n🎉 AUTO-START CONFIGURATION IS COMPLETE!")
        print("\nNext steps:")
        print("  1. Restart your computer")
        print("  2. Wait 2-5 minutes for n8n to start")
        print("  3. Access http://localhost:5678")
        print("  4. Verify workflows are accessible")
    elif results['checks_failed'] == 0:
        print("\n✅ AUTO-START IS CONFIGURED")
        print("\n⚠️  Review warnings above and address if needed")
    else:
        print("\n❌ AUTO-START CONFIGURATION INCOMPLETE")
        print("\nReview failed checks above and:")
        print("  • Complete Task Scheduler setup (Part 1)")
        print("  • OR install NSSM service (Part 2)")
        print("  • OR configure Docker restart policy (Part 4)")
    
    print("\n" + "=" * 70)

# Run verification for npm installation
print("Testing npm/Task Scheduler configuration...\n")
verify_autostart_config('npm')

print("\n\n")

# Test Docker configuration (uncomment if using Docker)
# print("Testing Docker configuration...\n")
# verify_autostart_config('docker')

## Exercises

### Exercise 1: Configure Task Scheduler Auto-Start (Easy)

**Task**: Set up automatic n8n startup using Task Scheduler.

**Steps**:
1. Create startup batch script at `C:\n8n\start-n8n.bat`
2. Test batch script by double-clicking it
3. Verify n8n starts successfully
4. Open Task Scheduler (`taskschd.msc`)
5. Create new task with startup trigger
6. Configure to run whether user is logged on or not
7. Set 1-minute delay after startup
8. Manually test task: Right-click → Run
9. Restart computer to verify auto-start
10. Confirm n8n is accessible after reboot

**Verification**: n8n starts automatically after computer restart without manual intervention.

**Deliverable**: Screenshot of Task Scheduler task properties and n8n running after reboot.

### Exercise 2: Install NSSM Windows Service (Medium)

**Task**: Convert n8n to a true Windows service using NSSM.

**Steps**:
1. Download NSSM from nssm.cc
2. Extract to `C:\tools\nssm`
3. Add to system PATH
4. Verify NSSM: `nssm --version`
5. Stop any running n8n instances
6. Use the generator above to create installation script
7. Run script as Administrator
8. Verify service installed: `services.msc`
9. Test service commands:
   - `nssm status n8n`
   - `nssm stop n8n`
   - `nssm start n8n`
10. Restart computer and verify auto-start
11. Check logs: `C:\n8n\logs\n8n-output.log`

**Verification**: n8n runs as Windows service, starts before user login, and restarts on failure.

**Deliverable**: Services.msc screenshot showing n8n service running, and log file showing successful startup.

### Exercise 3: Complete Auto-Start Test (Hard)

**Task**: Implement and thoroughly test auto-start configuration.

**Steps**:
1. Choose your preferred method (Task Scheduler, NSSM, or Docker)
2. Implement complete configuration
3. Document all configuration steps taken
4. Perform multiple restart tests:
   - Normal restart
   - Forced shutdown (simulated power failure)
   - Sleep/wake cycle (if laptop)
5. Measure startup time (boot to n8n accessible)
6. Test failure scenarios:
   - Stop n8n manually, verify it doesn't auto-restart
   - Simulate crash (kill process), verify restart
7. Configure monitoring/alerting (optional):
   - Email notification if n8n down
   - Workflow to check n8n health
8. Document troubleshooting steps for common issues
9. Create runbook for future reference

**Verification**: n8n starts reliably after all test scenarios, with documented startup times and failure recovery.

**Deliverable**: Complete test report with all scenarios, startup times, and configuration runbook.

## Summary

In this module, you learned four methods for auto-starting n8n on Windows:

### Method Comparison

1. **Task Scheduler**: 
   - Best for: Simple npm installations
   - Setup time: 10-15 minutes
   - Reliability: High
   - Complexity: Low

2. **NSSM (Non-Sucking Service Manager)**:
   - Best for: Production npm installations
   - Setup time: 20-30 minutes
   - Reliability: Very High
   - Complexity: Medium

3. **PM2 Process Manager**:
   - Best for: Cross-platform deployments
   - Setup time: 15-20 minutes
   - Reliability: Medium-High
   - Complexity: Medium

4. **Docker Restart Policies**:
   - Best for: Docker installations
   - Setup time: 5-10 minutes
   - Reliability: High
   - Complexity: Low

### Key Configuration Elements

✅ **Startup delay**: 1-2 minutes allows Windows to initialize  
✅ **Run whether user logged in**: Ensures startup without login  
✅ **Highest privileges**: Prevents permission issues  
✅ **Battery settings**: Allow start/run on battery (laptops)  
✅ **Restart on failure**: Automatic recovery from crashes  
✅ **Logging**: Track startup and identify issues  

### Testing Checklist

- [ ] Manual task/service start works
- [ ] n8n accessible after manual trigger
- [ ] Computer restart → n8n auto-starts
- [ ] Startup time acceptable (2-5 minutes)
- [ ] Workflows execute correctly after auto-start
- [ ] Logs show successful initialization
- [ ] Failure recovery works (if configured)

### Common Issues and Solutions

**Issue**: Task runs but n8n doesn't start  
**Solution**: Check batch script paths, verify Node.js in PATH

**Issue**: Service starts but stops immediately  
**Solution**: Check logs, verify environment variables, test n8n manually

**Issue**: Docker container doesn't restart  
**Solution**: Verify restart policy, check Docker Desktop auto-start

**Issue**: n8n accessible but workflows don't trigger  
**Solution**: Check timezone settings, verify webhook URLs, test triggers manually

### Recommendation Matrix

| Your Situation | Recommended Method |
|----------------|--------------------|
| npm + Simple setup | Task Scheduler |
| npm + Production | NSSM Service |
| npm + Monitoring needs | PM2 |
| Docker installation | Restart policy |
| Cross-platform (Windows/Linux) | PM2 or Docker |
| Maximum reliability | NSSM or Docker |

### What's Next

- **Module 09**: Updates and maintenance procedures (backup, update, rollback)
- **Module 10**: Production deployment strategies (laptop vs server decision)

### Additional Resources

- [Task Scheduler Documentation](https://docs.microsoft.com/en-us/windows/win32/taskschd/)
- [NSSM Official Site](https://nssm.cc/)
- [PM2 Documentation](https://pm2.keymetrics.io/)
- [Docker Restart Policies](https://docs.docker.com/config/containers/start-containers-automatically/)

### Remember

Auto-start configuration transforms n8n from a development tool to a production automation platform. Take time to test thoroughly—a reliable auto-start setup means your workflows keep running even during Windows updates, power outages, or unexpected reboots. The peace of mind is worth the 30-45 minutes of initial setup.